# Training & Evaluation  

## Importing Preprocessing Pipeline

In [96]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
# import matplotlib.pyplot as plt
# import plotly.express as px
# from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, FunctionTransformer
# from sklearn.metrics.pairwise import rbf_kernel

# from sklearn.compose import ColumnTransformer
# from scipy.signal import find_peaks
# from scipy.stats import gaussian_kde
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.utils.validation import check_array, check_is_fitted
# from sklearn.ensemble import IsolationForest
# from sklearn.cluster import KMeans


In [97]:
preprocessing_pipeline = None  # Define as a placeholder

%run "02_transform_data.ipynb"

In [98]:
preprocessing_pipeline

ColumnTransformer(transformers=[('bedrooms_per_room',
                                 Pipeline(steps=[('impute_ratios',
                                                  SimpleImputer(strategy='median')),
                                                 ('calculate_ratios',
                                                  FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7faaed9d13a0>,
                                                                      func=<function calculate_ratio at 0x7faaa6ad9620>)),
                                                 ('standard_scaler',
                                                  StandardScaler())]),
                                 ['total_bedrooms', 'total_roo...
                                                                      func=<function heavy_tail_distribution at 0x7faaa6ad9da0>,
                                                                      inverse_func=<ufunc 'expm1'>)),
                                                 ('standard_scaler',
                                                  StandardScaler())]),
                                 ['total_bedrooms', 'total_rooms', 'population',
                                  'households', 'median_income']),
                                ('categorical',
                                 Pipeline(steps=[('impute_categories',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encode_categories',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['ocean_proximity'])])

## Load Training Data

In [99]:
processed_data_path = Path("..", "data", "processed", "housing")

In [100]:
## read data
data = pd.read_csv(Path(processed_data_path, "train_set.csv"))

## Split Features & Labels

In [101]:
## before we create the pipeline lets split he training data into features and labels
df_features = data.drop("median_house_value", axis=1)
df_labels = data["median_house_value"].copy()

In [103]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   longitude              16512 non-null  float64
 1   latitude               16512 non-null  float64
 2   housing_median_age     16512 non-null  float64
 3   total_rooms            16512 non-null  float64
 4   total_bedrooms         16344 non-null  float64
 5   population             16512 non-null  float64
 6   households             16512 non-null  float64
 7   median_income          16512 non-null  float64
 8   ocean_proximity        16512 non-null  object 
 9   income_categories      16512 non-null  int64  
 10  population_categories  16512 non-null  int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 1.4+ MB


In [106]:
df_features.sample(10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_categories,population_categories
9288,-118.40,34.02,40.0,593.0,137.0,371.0,132.0,4.6932,<1H OCEAN,4,1
15714,-122.28,37.90,52.0,2003.0,250.0,658.0,244.0,10.0825,NEAR BAY,5,1
7056,-121.55,39.09,31.0,1728.0,365.0,1167.0,384.0,1.4958,INLAND,1,1
9830,-121.25,37.96,26.0,2205.0,478.0,1730.0,472.0,2.4866,INLAND,2,1
11731,-121.83,37.32,26.0,1125.0,210.0,943.0,214.0,4.8250,<1H OCEAN,4,1
1860,-122.24,38.01,16.0,2084.0,315.0,1154.0,307.0,6.0102,NEAR BAY,5,1
16058,-121.18,38.73,16.0,1584.0,264.0,613.0,226.0,6.0302,INLAND,5,1
2813,-118.48,34.04,47.0,1956.0,277.0,724.0,277.0,8.9616,<1H OCEAN,5,1
4430,-119.62,37.33,7.0,3389.0,621.0,1268.0,474.0,3.0224,INLAND,3,1
5005,-118.34,34.10,28.0,2223.0,752.0,1271.0,684.0,2.5434,<1H OCEAN,2,1


## Preprocessing Data

In [7]:
preprocessed_data = pd.DataFrame(preprocessing_pipeline.fit_transform(df_features), columns=preprocessing_pipeline.get_feature_names_out())

In [8]:
preprocessed_data.shape

(16512, 27)

In [9]:
## sanity check to see if we have empty values
preprocessed_data.isna().sum()

bedrooms_per_room__ratio                        0
rooms_per_household__ratio                      0
population_per_household__ratio                 0
multimodal_similarity__similarity_to_peak_17    0
multimodal_similarity__similarity_to_peak_26    0
multimodal_similarity__similarity_to_peak_35    0
multimodal_similarity__similarity_to_peak_52    0
cluster_similarity__similarity_to_cluster_0     0
cluster_similarity__similarity_to_cluster_1     0
cluster_similarity__similarity_to_cluster_2     0
cluster_similarity__similarity_to_cluster_3     0
cluster_similarity__similarity_to_cluster_4     0
cluster_similarity__similarity_to_cluster_5     0
cluster_similarity__similarity_to_cluster_6     0
cluster_similarity__similarity_to_cluster_7     0
cluster_similarity__similarity_to_cluster_8     0
cluster_similarity__similarity_to_cluster_9     0
log_pipeline__total_bedrooms                    0
log_pipeline__total_rooms                       0
log_pipeline__population                        0


## Training Linear Regression

### Training

In [10]:
from sklearn.linear_model import LinearRegression

linear_reg = make_pipeline(preprocessing_pipeline, LinearRegression())
linear_reg.fit(df_features, df_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7faaedee8180>,
                                                                                       func=<function calculate_ratio at 0x7faaedee85e0>)),
                                                                  ('standard_scaler',
                                                                   StandardSca...
                                                                                       inverse_func=<ufunc 'expm1'>)),
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('linearregression', LinearRegression())])

### Predictions

In [11]:
predictions = linear_reg.predict(df_features)
predictions[:5].round(2)

array([263335.29, 373549.57, 112870.95,  92243.99, 316741.99])

### Evaluations

#### Calculating Error Ratios

In [12]:
## lets compute error ratios
error_ratios = predictions[:5].round(-2) / df_labels.iloc[:5].values - 1
print(", ".join([f"{100 * ratio:.1f}%" for ratio in error_ratios]))

-42.5%, -22.8%, 11.0%, -4.1%, -12.5%


#### Calculating RMSE

In [13]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(df_labels, predictions)
rmse

66550.34931224315

Interpretation:
* On average the value predicted by our model has difference of `65550` then the actual value. 
* Based on project requirements since the output is fed to another ML model to predict whether we should invest in this area or not, a diff of `66K` might not give us any useful indicator. 
* Also since we are testing on training data, this difference indicates that we are underfitting the model. 


#### Calculating Relative RMSE

In [14]:
mean_price = df_labels.mean()
rmse_relative = rmse / mean_price
print(f"Relative RMSE: {rmse_relative:.2%}")


Relative RMSE: 32.25%


Interpretation
* Based on project requirement the current process is costly and time-consuming and estimates are off by more than 30%, our model is not performing better than the current process.
* We need to test other models, but before that lets run cross-validation to confirm our findings

#### Cross Validation

In [15]:
from sklearn.model_selection import cross_val_score

scores = -cross_val_score(linear_reg, df_features, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([66975.41917719, 66419.43069432, 64127.57598067, 75625.56219204,
       66741.20322301, 66746.47979067, 65562.44383051, 68892.13827255,
       66147.29254267, 67764.54064592])

Interpretation:
* Looks like our findings are correct, we are underfitting and getting similar RMSE values for all models
* Which means either our model is too simple or we don't have sufficient features. We need to try different models

## Random Forest

### Training

In [16]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing_pipeline, RandomForestRegressor(n_estimators=100, random_state=42))
forest_reg.fit(df_features, df_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7faaedee8180>,
                                                                                       func=<function calculate_ratio at 0x7faaedee85e0>)),
                                                                  ('standard_scaler',
                                                                   StandardSca...
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

### Predictions

In [17]:
predictions = forest_reg.predict(df_features)

In [18]:
predictions[:5].round(2)

array([433104.15, 478151.22, 106536.  , 101683.  , 370400.12])

### Evaluations

#### Calculating RMSE

In [19]:
rmse = root_mean_squared_error(df_labels, predictions)
rmse

17557.540624138303

Interpretations:
* RMSE is significanly less that `Linear Regression`
* One reason could be overfitting, we can confirm that using cross validation.
* If we are not overfitting, then we can use `GridSearch` to find the best params. 

#### Calcuating Relative RMSE

In [20]:
## calculate relative rmse
rmse_relative = rmse / mean_price
print(f"Relative RMSE: {rmse_relative:.2%}")

Relative RMSE: 8.51%


Interpretation:
* Seems like this model performs a lot better than our previous model, and than manual process. 
* If this model is not overfitting then it could be a candidate for production

#### Cross Validation

In [21]:
## cross validation
scores = -cross_val_score(forest_reg, df_features, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([46518.21599266, 47657.09029417, 46009.61776303, 47236.10087738,
       46343.77184697, 47318.70351497, 47534.58520817, 49669.7806786 ,
       47584.23865129, 46877.0007688 ])

In [22]:
scores.mean(), scores.std()

(47274.91055960402, 961.6083961071712)

Interpretations:
* We are getting different numbers, which are slightly better than `LinearRegression` but a lot worse than the model trained on whole training dataset. 
* Which means we are overfitting a bit when we train using the complete dataset. 

## Support Vector Regressor

### Training

In [23]:
## using support vector regression
from sklearn.svm import SVR

svm_reg = make_pipeline(preprocessing_pipeline, SVR(kernel="rbf"))
svm_reg.fit(df_features, df_labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7faaedee8180>,
                                                                                       func=<function calculate_ratio at 0x7faaedee85e0>)),
                                                                  ('standard_scaler',
                                                                   StandardSca...
                                                                                       func=<function heavy_tail_distribution at 0x7faaedf0cf40>,
                                                                                       inverse_func=<ufunc 'expm1'>)),
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('svr', SVR())])

### Predictions

In [24]:
predictions = svm_reg.predict(df_features)
predictions[:5].round(2)

array([179127.79, 179704.09, 178648.92, 178747.33, 179149.84])

Interpretations:
* This is worse than both the models, may be I need to pass separate hyperparameters. 
* May be we are underfitting, lets calculate evaluation metrics and then run cross validation to be sure. 

### Evaluations

#### Calculating RMSE

In [25]:
rmse = root_mean_squared_error(df_labels, predictions)
rmse

118239.52161044904

#### Calculating Relative RMSE

In [26]:
## relative rmse
rmse_relative = rmse / mean_price
print(f"Relative RMSE: {rmse_relative:.2%}")

Relative RMSE: 57.31%


#### Cross Validation

In [27]:
## cross validation
scores = -cross_val_score(svm_reg, df_features, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([120152.86799787, 121572.35148253, 115886.31068152, 117475.34228705,
       117464.73330846, 119809.06945733, 118817.52228608, 115841.63837712,
       115006.83130508, 120468.94123383])

Interpretation:
* Similar result, this might not be the right model for our data.  Or we might need hyperparameter tuning. 
* I think for now, lets focus on Random Forest hyper parameter tuning and see if we can find the right params. 

## Hyper Parameter Tuning

### Random Forest

* We can use the following params for hyper param tuning
    * n_estimators (Random Forest Regressor)
    * max_features (Random Forest Regressor)
    * n_clusters (ClusterSimilarityTransformer)
    * gamma (ClusterSimilarityTransformer)

#### Grid Search

In [ ]:
# hyper parameter tuning for random forest
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

random_forest_pipeline = Pipeline(
    [("preprocessing", preprocessing_pipeline), ("randomforestregressor", RandomForestRegressor())])

param_grid = [
    {"randomforestregressor__n_estimators": [3, 10, 30], 
     "randomforestregressor__max_features": [2, 4, 6, 8], 
     "randomforestregressor__bootstrap": [False], 
     "preprocessing__cluster_similarity__calculate_cluster__n_clusters": [2, 3, 4, 10],
     "preprocessing__cluster_similarity__calculate_cluster__gamma": [0.1, 0.5, 1.0, 2.0]}
]

grid_search = GridSearchCV(random_forest_pipeline, param_grid, cv=5,
                           scoring="neg_mean_squared_error", return_train_score=True)
grid_search.fit(df_features, df_labels)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('bedrooms_per_room',
                                                                         Pipeline(steps=[('impute_ratios',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('calculate_ratios',
                                                                                          FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7faaedee8180>,
                                                                                                              func=<function calculate_ratio at 0x7faaedee85e0>)),
                                                                                         ('stan...
             param_grid=[{'preprocessing__cluster_similarity__calculate_cluster__gamma': [0.1,
                                                                                          0.5,
                                                                                          1.0,
                                                                                          2.0],
                          'preprocessing__cluster_similarity__calculate_cluster__n_clusters': [2,
                                                                                               3,
                                                                                               4,
                                                                                               10],
                          'randomforestregressor__bootstrap': [False],
                          'randomforestregressor__max_features': [2, 4, 6, 8],
                          'randomforestregressor__n_estimators': [3, 10, 30]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [44]:
## best hyper parameters
grid_search.best_params_

{'preprocessing__cluster_similarity__calculate_cluster__gamma': 0.1,
 'preprocessing__cluster_similarity__calculate_cluster__n_clusters': 10,
 'randomforestregressor__bootstrap': False,
 'randomforestregressor__max_features': 6,
 'randomforestregressor__n_estimators': 30}

In [45]:
## best estimator
grid_search.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('bedrooms_per_room',
                                                  Pipeline(steps=[('impute_ratios',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('calculate_ratios',
                                                                   FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7faaedee8180>,
                                                                                       func=<function calculate_ratio at 0x7faaedee85e0>)),
                                                                  ('standard_scaler',
                                                                   StandardScaler(...
                                                                   StandardScaler())]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income']),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute_categories',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_categories',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  ['ocean_proximity'])])),
                ('randomforestregressor',
                 RandomForestRegressor(bootstrap=False, max_features=6,
                                       n_estimators=30))])

In [84]:
## function to convert params to dataframe
def summarize_search_resutls(mean_scores, params):
    ## scrore array
    grid_search_score = []

    for mean_score, params in zip(mean_scores, params):
        ## convert params to a dictionary
        params_dict = {}
        for key, value in params.items():
            params_dict[key] = value
        grid_search_score.append({"rmse": np.sqrt(-mean_score), **params_dict})  # store the results in a list

    return pd.DataFrame(grid_search_score)

In [ ]:
## create a dataframe of the results for better analysis
## evaluation scores
cvres = grid_search.cv_results_    
grid_search_score_df = summarize_search_resutls(cvres["mean_test_score"], cvres["params"])
grid_search_score_df.sort_values("rmse", ascending=True).head()

,rmse,preprocessing__cluster_similarity__calculate_cluster__gamma,preprocessing__cluster_similarity__calculate_cluster__n_clusters,randomforestregressor__bootstrap,randomforestregressor__max_features,randomforestregressor__n_estimators
44,43613.803190,0.1,10,False,6,30
41,44008.243668,0.1,10,False,4,30
92,44156.478368,0.5,10,False,6,30
89,44167.229997,0.5,10,False,4,30
140,44492.976978,1.0,10,False,6,30


Interpretation:
* So the best mean score is similar to what we had seen earlier. 
* Lets try randomized search to see if we get better params.

#### Randomized Search

In [49]:
# randomized search for hyper parameter tuning
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    "randomforestregressor__n_estimators": randint(low=1, high=200),
    "randomforestregressor__max_features": randint(low=1, high=8),
    "preprocessing__cluster_similarity__calculate_cluster__n_clusters": randint(low=10, high=100),
}

rnd_search = RandomizedSearchCV(random_forest_pipeline, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring="neg_mean_squared_error", random_state=42)
rnd_search.fit(df_features, df_labels)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(transformers=[('bedrooms_per_room',
                                                                               Pipeline(steps=[('impute_ratios',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('calculate_ratios',
                                                                                                FunctionTransformer(feature_names_out=<function calculate_ratio_feature_names at 0x7faaedee8180>,
                                                                                                                    func=<function calculate_ratio at 0x7faaedee85e0>)),...
                   param_distributions={'preprocessing__cluster_similarity__calculate_cluster__n_clusters': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7faaae957110>,
                                        'randomforestregressor__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7faaae956fc0>,
                                        'randomforestregressor__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7faaae956f60>},
                   random_state=42, scoring='neg_mean_squared_error')

In [50]:
rnd_search.best_params_

{'preprocessing__cluster_similarity__calculate_cluster__n_clusters': 39,
 'randomforestregressor__max_features': 6,
 'randomforestregressor__n_estimators': 130}

In [86]:
cvres = rnd_search.cv_results_
grid_search_score_df = summarize_search_resutls(cvres["mean_test_score"], cvres["params"])
grid_search_score_df.sort_values("rmse", ascending=True).head()

,rmse,preprocessing__cluster_similarity__calculate_cluster__n_clusters,randomforestregressor__max_features,randomforestregressor__n_estimators
6,41530.693224,39,6,130
2,41912.065568,92,7,75
9,42088.570362,31,5,89
8,42392.694677,42,4,58
3,42453.006781,84,5,100


Interpretation:
* Interesting this is slightly better than before, not a huge difference.
* Lets see if we can reduce the number of columns and just focus on important ones for training. 

In [63]:
## mapping important feature names to their importance scores
feature_importances = rnd_search.best_estimator_.steps[1][1].feature_importances_
important_feature_list = pd.DataFrame({"feature": rnd_search.best_estimator_.steps[0][1].get_feature_names_out(), "importance": feature_importances}).sort_values("importance", ascending=False)
important_feature_list


,feature,importance
50,log_pipeline__median_income,0.169170
0,bedrooms_per_room__ratio,0.062293
52,categorical__ocean_proximity_INLAND,0.056604
1,rooms_per_household__ratio,0.053629
2,population_per_household__ratio,0.044949
42,cluster_similarity__similarity_to_cluster_35,0.037642
23,cluster_similarity__similarity_to_cluster_16,0.032047
41,cluster_similarity__similarity_to_cluster_34,0.025910
10,cluster_similarity__similarity_to_cluster_3,0.023296
22,cluster_similarity__similarity_to_cluster_15,0.021597


Interpretation:
* Interesting there are lot of unimportant features, that we can get rid of and may be improve model performance. 
* Lets try and create a list of features to remove and see if we can improve the model
* Lets remove features with importance < 0.05

In [ ]:
## features greater than or equal to 0.05
important_feature_list[important_feature_list["importance"] >= 0.05]

,feature,importance
50,log_pipeline__median_income,0.169170
0,bedrooms_per_room__ratio,0.062293
52,categorical__ocean_proximity_INLAND,0.056604
1,rooms_per_household__ratio,0.053629


* Interesting we only have 4 features that contributes most to this model.
* This salso indicates that our `hyperparameters` related to geo clusters do not contribute too much to model training. 
* Lets use the complete training dataset, with just these 4 features and see if overfitting is reduced. 
* We will also save the best param model, because for now this is the best model we have and we might end up using it in test/prod. 


#### Saving Model

In [89]:
## Keep this code commented to avoid accidental overwriting of the model
## saving the model to disk
# from joblib import dump

# ## model path
# model_path = Path("..", "models")

# dump(rnd_search.best_estimator_, Path(model_path, "random_forest_model_v0_1.joblib"))

In [75]:
preprocessed_data_reduced = preprocessed_data[important_feature_list[important_feature_list["importance"] >= 0.05]["feature"].values]
preprocessed_data_reduced

,log_pipeline__median_income,bedrooms_per_room__ratio,categorical__ocean_proximity_INLAND,rooms_per_household__ratio
0,-1.078547,1.846624,0.0,-0.866027
1,1.231761,-0.508121,0.0,0.024550
2,-0.792464,-0.202155,1.0,-0.041193
3,-0.935308,-0.149006,1.0,-0.034858
4,-0.018667,0.963208,0.0,-0.666554
...,...,...,...,...
16507,0.628497,0.804368,0.0,-0.452111
16508,-0.680615,-0.192328,1.0,0.036792
16509,0.291670,-0.242492,0.0,-0.109987
16510,0.337576,0.259775,0.0,-0.360937


#### Reduced Features Training

Plan:
* So now we'll train on complete training dataset, with tuned hyper params and reduced feature list
* The hyperparameters will be `max_features = 6` and `n_estimators = 130`
* Assumption is if we get performance similar or slightly better than randomized grid, we are not overfitting the model and we can use it for production. 

In [90]:
## removing preprocessing since we have the preprocessed data
random_forest_pipeline = make_pipeline(RandomForestRegressor(n_estimators=130, max_features=6, random_state=42))

## fit the model
random_forest_pipeline.fit(preprocessed_data_reduced, df_labels)

Pipeline(steps=[('randomforestregressor',
                 RandomForestRegressor(max_features=6, n_estimators=130,
                                       random_state=42))])

In [91]:
## predictions
predictions = random_forest_pipeline.predict(preprocessed_data_reduced)
predictions[:5].round(2)

array([358842.31, 399844.62, 102266.15,  91250.77, 319537.69])

In [92]:
## rmse
rmse = root_mean_squared_error(df_labels, predictions)
rmse

27622.39252701624

* Interesting, the RMSE number is better than Random Search, and feels like less overfitting than before. But we'll have to do cross validation to confirm.

In [93]:
## cross validation
scores = -cross_val_score(random_forest_pipeline, preprocessed_data_reduced, df_labels, scoring="neg_root_mean_squared_error", cv=10)
scores

array([76483.50604172, 73089.38710089, 71217.10922862, 72613.50851974,
       74173.06529015, 75775.83707397, 74418.93059679, 74700.74346543,
       74039.56440709, 75664.63157784])

* Ok this performs worse than previous cross validation of random forest with mean `rmse` of `~47K`
* So for now, the best model we have is the random search model. Obviously we haven't tested all possible models, and that is because of scope of this project and my limited knowledge in ML. So we'll verify how random search model does on test data and proceed to next part which is creating a simple API.

In [94]:
## testing random search against the test set

## read test data
test_data = pd.read_csv(Path(processed_data_path, "test_set.csv"))

## split the test data into features and labels

test_features = test_data.drop("median_house_value", axis=1)
test_labels = test_data["median_house_value"].copy()

## preprocess the test data
preprocessed_test_data = pd.DataFrame(preprocessing_pipeline.transform(test_features), columns=preprocessing_pipeline.get_feature_names_out())

## predictions using rnd_search
predictions = rnd_search.best_estimator_.predict(test_features)

## rmse
rmse = root_mean_squared_error(test_labels, predictions)

## relative rmse

mean_price = test_labels.mean()
rmse_relative = rmse / mean_price
print(f"RMSE: {rmse}, Relative RMSE: {rmse_relative:.2%}")


RMSE: 41479.02032719327, Relative RMSE: 19.85%


* So we have a model, which has `Relative RMSE 19.85%` which is a lot better than current manual process where the estimates are off by more than 30%.
* The model is approx `33%` better than current manual process.
* We are going to use this as our production model and for the API.